## The Cat API

In [ ]:
# !pip install requests

In [6]:
catkey = "foobar"

In [13]:
import requests

def get_random_cat_image(api_key):
    url = "https://api.thecatapi.com/v1/images/search"
    headers = {
        "x-api-key": api_key
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Extract the URL of the cat image from the response
        image_url = data[0]["url"]
        print(response.text)
        return image_url
    else:
        print("Failed to fetch cat image:", response.text)
        return None

# Replace 'YOUR_API_KEY' with your actual API key from The Cat API
api_key = catkey
image_url = get_random_cat_image(api_key)
if image_url:
    print("Random cat image URL:", image_url)
else:
    print("Failed to fetch a random cat image.")


[{"breeds":[],"id":"27l","url":"https://cdn2.thecatapi.com/images/27l.jpg","width":680,"height":455}]
Random cat image URL: https://cdn2.thecatapi.com/images/27l.jpg


## Classify that picture

In [ ]:
# !pip install tensorflow keras pillow

In [4]:
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import requests
from io import BytesIO

In [34]:
def classify(img_url):
    # Download the image from the URL
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))

    # Convert the image to a numpy array
    x = image.img_to_array(img.resize((224, 224)))
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Run the image through the model
    preds = model.predict(x)

    # Decode the results into a list of tuples (class, description, probability)
    return decode_predictions(preds, top=1)[0][0]

In [35]:
# Classify an image
result = classify(image_url)

# Check if the image is a cat
if result[1] == 'tabby' or result[1] == 'tiger_cat':
    print("It's a cat!")
else:
    print("It's not a cat!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
It's not a cat!


### fix the error

In [36]:
# Load the pre-trained model
model = MobileNetV2(weights='imagenet')

def classify(img_url):
    # Download the image from the URL
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))

    # Convert the image to a numpy array
    x = image.img_to_array(img.resize((224, 224)))
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Run the image through the model
    preds = model.predict(x)

    # Decode the results into a list of top-5 classes
    pred_info = decode_predictions(preds, top=5)[0]
    return [i[1] for i in pred_info]

In [37]:
# Classify an image
results = classify(img_url=image_url)

# Check if the image is a cat
if "tabby" in results or any("_cat" in item for item in results):
    print("It's a cat!")
else:
    print("It's not a cat!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
It's a cat!
